In [39]:
import csv
import json
import time
import datetime
import pandas as pd
from tqdm import tqdm
import itertools

import requests


### get top 250 coins' id

In [40]:
#gets data from coingecko
response = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page=1&sparkline=false')
response = response.json()

coin_list = []
for coin in response:
    coin_list.append({'id':coin['id'], 'symbol':coin['symbol'], 'name':coin['name']})

for c in coin_list[:3]:
    print(c['id'])


bitcoin
ethereum
tether


In [47]:
print(coin_list[4])

{'id': 'binancecoin', 'symbol': 'bnb', 'name': 'BNB'}


In [59]:
def date_to_unix_time(date, spliter='-'):
    ymd = date.split(spliter)
    my_datetime = datetime.date(int(ymd[0]), int(ymd[1]), int(ymd[2]))
    return int(time.mktime(my_datetime.timetuple()))

'''
1 day from current time = 5 minute interval data
1 ~ 90 days from current time = hourly data
above 90 days from current time = daily data (00:00 UTC)
'''
start_date = '2022-01-01'
end_date = "2022-12-01"

def get_one_coin_json(coin_id, _from, _to):
    response = requests.get('https://api.coingecko.com/api/v3/coins/' + coin_id + '/market_chart/range?vs_currency=usd&from='+ _from + '&to=' + _to)
    return response.json()

def wait_api(times=1):
    sec = 60
    for i in range(sec+1):
        print('waiting API({})....  ({:d}s)'.format(times, sec - i), end="\r")
        time.sleep(1)

### get price, market_caps, volume of all coins


> every 5 api calls needs 1 minute break in free version <br>
> 250 coins need ~50 minutes


In [53]:
#get all coins data

alt_caps_data = []
btc_data = {}
date_list = []

api_limit = 50
retry_limit = 3
count = 1

print('downloading:')
for idx, coin in tqdm(enumerate(coin_list), total=len(coin_list)):
    if((idx + 1) / api_limit >= count):
        count += 1
        wait_api()
     
    data = get_one_coin_json(coin['id'], str(date_to_unix_time(start_date)), str(date_to_unix_time(end_date))) 
    retry_times = 0
    while 'market_caps' not in data:
        if retry_times >= retry_limit:
            raise Exception("Retry Limit Exceeded! Check https://status.coingecko.com/ for API Status")
        retry_times += 1
        wait_api(retry_times)
        data = get_one_coin_json(coin['id'], str(date_to_unix_time(start_date)), str(date_to_unix_time(end_date))) 

    if(coin['id'] != 'bitcoin'):   
        alt_caps_data.append([mc[1] for mc in data['market_caps']])
    else:
        def append_date_list_and_return_num(li):
            # https://stackoverflow.com/questions/9744775/how-to-convert-integer-timestamp-into-a-datetime
            date_list.append(datetime.datetime.fromtimestamp(li[0] / 1e3))
            return li[1]
        btc_data['prices'] = [append_date_list_and_return_num(d) for d in data['prices']]
        btc_data['market_caps'] = [d[1] for d in data['market_caps']]
        btc_data['total_volumes'] = [d[1] for d in data['total_volumes']]
        
    # dflist = pd.DataFrame({'prices', 'market_caps', 'total_volumes'})
    '''
    {
        "prices": [
        [
        1641600000000,
        3201.79457434796
        ],
        [
        1641686400000,
        3101.0409016141402
        ],
    }
    '''



downloading:


  2%|▏         | 5/250 [00:02<02:51,  1.43it/s]

waiting API(1)....  (52s)

  2%|▏         | 5/250 [00:11<09:46,  2.40s/it]


KeyboardInterrupt: 

### process data and save to csv

In [ ]:
# process data to csv

alt_coin_num = len(alt_caps_data)
date_num = len(alt_caps_data[0])
alt_caps_sum = [0] * date_num

for date_idx, coin_idx in itertools.product(range(date_num), range(alt_coin_num)):
    alt_caps_sum[date_idx] += alt_caps_data[coin_idx][date_idx]

btc_data['BTCD'] = [ btc_data['market_caps'][i] / (alt_caps_sum[i]+btc_data['market_caps'][i]) * 100 for i in range(date_num) ]

df = pd.DataFrame(btc_data, index=date_list)
# print(df)

df.to_csv('Bitcoin_{}_{}_Price_MarketCap_Volume_BTCD.csv'.format(start_date, end_date))

                           prices   market_caps  total_volumes       BTCD
2022-01-01 08:00:00  46319.651088  8.761929e+11   2.589535e+10  62.837424
2022-01-02 08:00:00  47816.077676  9.045519e+11   1.890459e+10  63.117007
2022-01-03 08:00:00  47387.212168  8.975361e+11   3.375613e+10  62.639053
2022-01-04 08:00:00  46531.140861  8.803302e+11   2.189002e+10  62.553306
2022-01-05 08:00:00  45938.024272  8.762427e+11   2.489599e+10  62.174884
...                           ...           ...            ...        ...
2022-11-26 08:00:00  16524.449008  3.179199e+11   1.893302e+10  60.203453
2022-11-27 08:00:00  16456.156062  3.162069e+11   1.824302e+10  60.027793
2022-11-28 08:00:00  16459.149544  3.164877e+11   1.655182e+10  60.119822
2022-11-29 08:00:00  16222.210729  3.118110e+11   2.456606e+10  60.169491
2022-11-30 08:00:00  16441.979978  3.161709e+11   2.466304e+10  59.848368

[334 rows x 4 columns]
